In [1]:
import pandas as pd
import urllib, urllib.request
import requests
import time
import json

In [10]:
testmeta = pd.read_csv('test_metadata.tsv', sep = '\t')
testmeta.head()

,journal,authors,jstordoi,pubdoi,title,canfindjdoi,canfindpdoi,canfindtitle,date
0,American Economic Review,"['Michael D. Bauer', 'Glenn D. Rudebusch']",10.2307/26921585,10.1257/aer.20171822,Interest Rates under Falling Stars,NaN,NaN,NaN,2020
1,Modern Philology,['Elmer Edgar Stoll'],10.2307/435244,10.1086/388769,"A New ""Reading"" of ""Othello""",NaN,NaN,NaN,1948
2,Critical Inquiry,['William J. Rankin'],10.1086/655212,10.1086/655212,"The Epistemology of the Suburbs: Knowledge, Pr...",NaN,NaN,NaN,2010
3,PMLA,['GILLIAN SILVERMAN'],10.2307/26158813,10.1632/pmla.2016.131.2.307,Neurodiversity and the Revision of Book History,NaN,NaN,NaN,2016


In [11]:
def record_result(res, searchtitle, searchtype):
    if 'title' in res:
        title=res["title"]
        found = True
    else:
        title = 'Title Not Provided'
        found = False
    citation_count = 0
    cited_ids = []
    if 'citations' in res:
        for c in res["citations"]:
            cited_ids.append(c["paperId"])
        citation_count=len(cited_ids)
    if found:
        print(searchtitle + ' found by ' + searchtype)
        print('with ' + str(citation_count) + ' citations.')
        print
    else:
        print(searchtitle + ' not found by ' + searchtype)

for idx, row in testmeta.iterrows():
    # let's start with JSTOR doi
    jdoi = row['jstordoi']
    searchtitle = row['title']
    
    print()
    url="https://api.semanticscholar.org/graph/v1/paper/DOI:" + jdoi + "?fields=title,publicationDate,authors,s2FieldsOfStudy,citations,citations.authors"
    response = requests.get(url)
    try:
        resplen = len(response.json())
    except:
        missed.append(url)
        print(DOI)
        idx += 1
        #waits when timeout
        time.sleep(0.1)
        continue
    result =response.json()
    if 'error' in result:
        print('J')
        print(result)
    else:
        record_result(result, searchtitle, 'JSTOR doi')
    
    # now publisher ID
    pdoi = row['pubdoi']
    url="https://api.semanticscholar.org/graph/v1/paper/DOI:" +pdoi + "?fields=title,publicationDate,authors,s2FieldsOfStudy,citations,citations.authors"
    response = requests.get(url)
    try:
        resplen = len(response.json())
    except:
        missed.append(url)
        print(DOI)
        idx += 1
        #waits when timeout
        time.sleep(0.1)
        continue
    result =response.json()
    if 'error' in result:
        print('P')
        print(result)
    else:
        record_result(result, searchtitle, 'publisher doi')
        


J
{'error': 'Paper with id DOI:10.2307/26921585 not found'}
P
{'error': 'Paper with id DOI:10.1257/aer.20171822 not found'}

J
{'error': 'Paper with id DOI:10.2307/435244 not found'}
A New "Reading" of "Othello" found by publisher doi
with 0 citations.

The Epistemology of the Suburbs: Knowledge, Production, and Corporate Laboratory Design found by JSTOR doi
with 13 citations.
The Epistemology of the Suburbs: Knowledge, Production, and Corporate Laboratory Design found by publisher doi
with 13 citations.

J
{'error': 'Paper with id DOI:10.2307/26158813 not found'}
Neurodiversity and the Revision of Book History found by publisher doi
with 10 citations.


In [32]:
def search_papers(title, year=None):
    title = title.replace('"', '')
    url="https://api.semanticscholar.org/graph/v1/paper/search?query=title:" + title + '&fields=title,authors,publicationDate'
    response = requests.get(url)
    
    if response.status_code == 200:
        jsonobj = response.json()
        if 'data' in jsonobj:
            papers = jsonobj['data']
        else:
            print(jsonobj)
            
        print(papers)
        for paper in papers:
            foundtitle = paper['title']
            if not paper['publicationDate'] is None:
                foundyear = int(paper['publicationDate'][0:4])
                found = False
            else:
                foundyear = 0
            if foundtitle.lower() == title.lower() and foundyear == year:
                print('Found')
                found = True
        if not found:
            print('Not found')
                
    else:
        print(f'Failed to retrieve papers: {response.status_code}')
        print(response)


for idx, row in testmeta.iterrows():
    # let's start with JSTOR doi
    jdoi = row['jstordoi']
    searchtitle = row['title']
    pubyear = row['date']
    print(searchtitle, pubyear)
    search_papers(searchtitle, pubyear)  
    

Interest Rates under Falling Stars 2020
[{'paperId': '1d1a19428ef07b82f5fbbaf8828448a2fcc3089f', 'title': 'Interest Rates Under Falling Stars', 'publicationDate': '2017-07-10', 'authors': [{'authorId': '145421946', 'name': 'M. Bauer'}, {'authorId': '65729671', 'name': 'Glenn D. Rudebusch'}]}, {'paperId': '65fbfa65de32ddede715da5b50f86ce1a1b4444a', 'title': 'Online Appendix for “Interest Rates Under Falling Stars”', 'publicationDate': None, 'authors': [{'authorId': '145421946', 'name': 'M. Bauer'}, {'authorId': '65729671', 'name': 'Glenn D. Rudebusch'}]}, {'paperId': '9f0f8524536cdb243badb8796c6fe8a937a48f1e', 'title': 'Unintended Effects of Macro-Prudential Policy on Real Interest Rates and Liquidity Traps', 'publicationDate': None, 'authors': [{'authorId': '1741302618', 'name': 'Alejandro Van der Ghote'}]}, {'paperId': '9092429ed30bf763dd80104d02d376ca1e8e7ca0', 'title': 'Monetary policy special features in the context of low interest rates', 'publicationDate': None, 'authors': [{'aut

UnboundLocalError: local variable 'papers' referenced before assignment

In [72]:
newlitmeta.at[idx, 'doi']

'10.2307/512209'

In [73]:
idx

0

In [74]:
DOI

'10.2307/461288'

In [69]:
ids

['10.2307/512209',
 '10.2307/437950',
 '10.2307/461288',
 '10.2307/3661498',
 '10.2307/3718589',
 '10.2307/2872855',
 '10.2307/3723455',
 '10.2307/512967',
 '10.2307/40587680',
 '10.2307/438705',
 '10.2307/26158877',
 '10.2307/4501647',
 '10.2307/458780',
 '10.2307/459148',
 '10.2307/3736691',
 '10.2307/3728328',
 '10.2307/514110',
 '10.2307/433444',
 '10.2307/3717854',
 '10.2307/434174',
 '10.1086/598693',
 '10.2307/27654644',
 '10.2307/432736',
 '10.2307/437350',
 '10.2307/1261371',
 '10.2307/3730409',
 '10.2307/3715100',
 '10.2307/44016509',
 '10.2307/458990',
 '10.2307/468576',
 '10.2307/509276',
 '10.2307/457562',
 '10.2307/438895',
 '10.2307/3713225',
 '10.2307/3736356',
 '10.2307/511338',
 '10.2307/44016445',
 '10.1086/588108',
 '10.2307/20057704',
 '10.2307/3713138',
 '10.2307/510839',
 '10.2307/438632',
 '10.2307/3734330',
 '10.2307/1343470',
 '10.2307/3661532',
 '10.2307/3731247',
 '10.2307/508839',
 '10.2307/3727545',
 '10.2307/457644',
 '10.2307/2873185',
 '10.2307/436676',

In [60]:
DOI

,journal,year,authors,title,language,wordcount,doctype,doi,citation_counts,S2titles,S2years
0,The Review of English Studies,1969,[C. F. Williamson],Review Article,[eng],572,article | book-review,10.2307/512209,0,NA,0
1,Modern Philology,1979,[David Halliburton],Review Article,[eng],1863,article | book-review,10.2307/437950,0,NA,0
2,PMLA,1967,[Jean-Jacques Demorest],Pascal et le déséquilibre,[fre],4999,article | research-article,10.2307/461288,0,NA,0
3,The Review of English Studies,2003,[Robert Douglas-Fairhurst],Review Article,[eng],1140,article | book-review,10.2307/3661498,0,NA,0
4,The Modern Language Review,1951,[R. Weiss],Review Article,[eng],669,article | book-review,10.2307/3718589,0,NA,0


In [23]:
ret

[[None,
  'The Revisions of "Piers Plowman"',
  ['Art'],
  ['Mabel Day'],
  ['5345b72b54964d4009f83bfdc48a746c58a68d84'],
  ['S. Trigg'],
  1]]

In [68]:
out=pd.DataFrame(ret)

In [69]:
out.columns=["date", "title", "tags", "authors", "cited_ids", "cited_authors", "citation_count"]

In [71]:
out.to_csv("semantic_meta.csv")